This here sheet is going to try to rotate two images so they can subtracted ....with AIJ?  I think this was just a draft for the Image Rotate rroutine.

Using astroalign: https://astroalign.quatrope.org/en/latest/

In [1]:
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits

In [2]:
import numpy as np
np.version.version

'1.26.3'

In [3]:
import astroalign as aa
aa.__version__

'2.5.1'

This script rotates the source to match the target.

In [4]:
source_name='/run/media/cduston/MCARG/AnalyzedSources/SN2023ctn/Stacked/rotated/SUM_Aligned_20240119_B.fits'
target_name='/run/media/cduston/MCARG/AnalyzedSources/SN2023ctn/Stacked/rotated/SUM_Aligned_20230310_B.fits'

In [5]:
file_source = get_pkg_data_filename(source_name)
file_target = get_pkg_data_filename(target_name)

In [6]:
#file_target = fits.util.get_testdata_filepath('/home/cduston/GoogleDrive/Research/MCARG/Novae/SN2023ctn/Stacked/SUM_Aligned_20230310_I.fits')
#file_source = fits.util.get_testdata_filepath('/home/cduston/GoogleDrive/Research/MCARG/Novae/SN2023ctn/Stacked/SUM_Aligned_20240119_I.fits')

Load FITS data...

In [7]:
hdu_source = fits.open(file_source)
hdu_source.info()

Filename: /run/media/cduston/MCARG/AnalyzedSources/SN2023ctn/Stacked/rotated/SUM_Aligned_20240119_B.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      79   (2250, 1800)   float32   


In [8]:
hdu_target = fits.open(file_target)
hdu_target.info()

Filename: /run/media/cduston/MCARG/AnalyzedSources/SN2023ctn/Stacked/rotated/SUM_Aligned_20230310_B.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      79   (2250, 1800)   float32   


couple of checks...

In [9]:
target=hdu_target[0].data
source=hdu_source[0].data
print(type(target))
print(target.shape)

<class 'numpy.ndarray'>
(1800, 2250)


In [10]:
print('Min:', np.min(source))
print('Max:', np.max(source))
print('Mean:', np.mean(source))
print('Stdev:', np.std(source))

Min: 176.27298
Max: 216558.36
Mean: 965.6241
Stdev: 2753.698


These lines are to deal with some kind of small/large emden order, see this: 

https://github.com/quatrope/astroalign/issues/70

In [11]:
target = target.byteswap().newbyteorder()
source = source.byteswap().newbyteorder()

This is going to rotate source to match target. The detection sigma was from the example, so is pretty random.

In [12]:
img_aligned, footprint = aa.register(source, target, detection_sigma=3.0)

Now copy data into a new file.

In [13]:
hdu = fits.PrimaryHDU(img_aligned)
#hdul = fits.HDUList([hdu])

In [14]:
hdr = hdu_source[0].header
hdr['HISTORY'] = 'Rotated to match other images in the stack'
hdu.header=hdr

In [15]:
#hdul.writeto('Rotated_I.fits')
hdu.writeto('SUM_Aligned_Rotated_20240119_B.fits',overwrite=True)